In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#read excel file into pandas dataframe
df=pd.read_excel('trial_redun_temp_humid.xlsx',engine='openpyxl')

In [2]:
#Setting Predictors
X=df.drop(columns=['AQI','Datetime','Level of Health Concern']).values

#Setting Target
Y=df['AQI'].values

#Splitting dataset into Train and Test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
print(X_test)


[[6 7 2019 ... False 72.9 74.5]
 [11 27 2019 ... False 57.4 73.6]
 [6 5 2019 ... False 73.6 66.1]
 ...
 [9 24 2019 ... False 78.4 31.5]
 [10 8 2019 ... False 60.8 74.0]
 [1 25 2019 ... False 34.9 59.0]]


In [3]:
# Building  Random Forest Classifier
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state =42 )
rfr.fit(X_train, Y_train)

RandomForestRegressor(random_state=42)

In [4]:
# Evaluating on Training set
rfr_pred_train = rfr.predict(X_train)
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
print(np.sqrt(mean_squared_error(Y_train,rfr_pred_train)))
print(r2_score(Y_train,rfr_pred_train))


# Evaluating on Test set
rfr_pred_test = rfr.predict(X_test)
print(np.sqrt(mean_squared_error(Y_test,rfr_pred_test)))
print(r2_score(Y_test,rfr_pred_test))

2.6321801152052537
0.9837024283278416
6.68424317962608
0.8930231777337199


In [5]:
#Saving to S3
import tempfile
import joblib
import boto3
s3 = boto3.resource('s3')
bucket='digitwin-airqualitydata'
key='EPA_data/EPA_rfr_temp_humid.sav'

# WRITE
with tempfile.TemporaryFile() as fp:
    joblib.dump(rfr, fp)
    fp.seek(0)
    s3.Bucket(bucket).put_object(Key= key, Body=fp.read())
